In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import requests

In [ ]:
def get_expiry_list(index):
    url = f"https://live.markethound.in/api/history/expiries?index={index}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data")
        return None


index = "FINNIFTY"

In [ ]:

# Step 1: Data Collection
def get_historical_data(index, expiry_date):
    url = f"https://live.markethound.in/api/history/decay?name={index}&expiry={expiry_date}&dte=0"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data")
        return None


index = "FINNIFTY"
expiry_date = "2024-03-05T00:01:01.000Z"
historical_data = get_historical_data(index, expiry_date)


In [ ]:
# Step 2: Feature Engineering
# Extract relevant features from the collected data

def extract_features(data):
    features = []
    for entry in data:
        feature_vector = [
            entry['open'],
            entry['high'],
            entry['low'],
            entry['close'],
            entry['expiry'],
            entry['date']
            # Add more features as needed
        ]
        features.append(feature_vector)
    return features

In [ ]:
# Step 3: Data Preprocessing
# Handle missing values, normalize or scale the features, split data into train and test sets
def preprocess_data(features):
    df = pd.DataFrame(features, columns=['Open', 'High', 'Low', 'Close', 'Expiry', 'Date'])
    # X = df.drop(columns=['Close'])
    # X = df
    # y = df['Close']
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return df

In [ ]:
# Putting it all together
if __name__ == "__main__":

    expiry_list = get_expiry_list(index)
    print(expiry_list['result'])

    for ex in expiry_list['result']:
        historical_data = get_historical_data(index, expiry_date)
        # print(historical_data['result'])
        features = extract_features(historical_data['result'])

['2024-03-12T00:00:00.000Z', '2024-03-05T00:00:00.000Z', '2024-02-27T00:00:00.000Z', '2024-02-20T00:00:00.000Z', '2024-02-13T00:00:00.000Z', '2024-02-06T00:00:00.000Z', '2024-01-30T00:00:00.000Z', '2024-01-23T00:00:00.000Z', '2024-01-16T00:00:00.000Z', '2024-01-09T00:00:00.000Z', '2024-01-02T00:00:00.000Z', '2023-12-26T00:00:00.000Z', '2023-12-19T00:00:00.000Z', '2023-12-12T00:00:00.000Z', '2023-12-05T00:00:00.000Z', '2023-11-28T00:00:00.000Z', '2023-11-21T00:00:00.000Z', '2023-11-13T00:00:00.000Z', '2023-11-07T00:00:00.000Z', '2023-10-31T00:00:00.000Z', '2023-10-23T00:00:00.000Z', '2023-10-17T00:00:00.000Z', '2023-10-10T00:00:00.000Z', '2023-10-03T00:00:00.000Z', '2023-09-26T00:00:00.000Z', '2023-09-18T00:00:00.000Z', '2023-09-12T00:00:00.000Z', '2023-09-05T00:00:00.000Z', '2023-08-29T00:00:00.000Z', '2023-08-22T00:00:00.000Z', '2023-08-14T00:00:00.000Z', '2023-08-08T00:00:00.000Z', '2023-08-01T00:00:00.000Z', '2023-07-25T00:00:00.000Z', '2023-07-18T00:00:00.000Z']
     Open    High  

In [37]:
data=preprocess_data(features).filter(['Open'])

#Convert to np array
dataset=data.values

#Get the number of rows to train the model on
#math.ceil to round up

training_data_len = math.ceil(len(dataset)*.8)


#print(dataset)
training_data_len

NameError: name 'df' is not defined

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

In [ ]:
train_data = scaled_data[0:training_data_len,:]

x_train=[]
y_train=[]

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if i<=60:
        print(x_train)
        print(y_train)
        print()

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [ ]:

model=Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))

model.add(LSTM(50, return_sequences=False))

model.add(Dense(25))

model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, batch_size=1,epochs=3)

In [ ]:
test_data=scaled_data[training_data_len-60: , : ]

x_test=[]
y_test=dataset[training_data_len:,:]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
x_test=np.array(x_test)

In [ ]:
x_test=np.reshape(x_test,(x_test.shape[0], x_test.shape[1], 1))

In [ ]:
predictions=model.predict(x_test)
predictions=scaler.inverse_transform(predictions)

In [ ]:
rmse =np.sqrt(np.mean(predictions-y_test)**2)
rmse